# Toy model

This toy model has the purpose of testing the VEGAS algorithm in order to compute a discret sum.

I consider a function $f$ defined as:

$$
f(x_0, y_0, z_0) = \sum\limits_{x = 0}^{10} \sum\limits_{y = 0}^{10} \sum\limits_{z = 0}^{10} e^{-\frac{1}{2} \left( x - x_0 \right)^2} e^{-\frac{1}{2} \left( y - y_0 \right)^2} e^{-\frac{1}{2} \left( z - z_0 \right)^2} \sin \left( xy \right) \cos \left( z \right)
$$

In [1]:
function f(x0, y0, z0)
    
    result = 0.0
   
    for x=0:10, y=0:10, z=0:10
    
        result += exp(-(1/2)*(x - x0)^2)*exp(-(1/2)*(y - y0)^2)*exp(-(1/2)*(z - z0)^2)*sin(x*y)*cos(z)
        
    end
        
    return result
    
end

f (generic function with 1 method)

## Exact result

Let's first compute the exact value of the function for some values of $(x_0, y_0, z_0)$

In [6]:
@time f(2,6,10)

  0.000041 seconds


0.692056321795933

In [8]:
@time f(0,8,10)

  0.000039 seconds


-1.2701664008437514

In [9]:
@time f(7,2.4,3)

  0.000040 seconds


-1.0645141883730707

## Standard Monte Carlo

Let's now construct a standard Monte Carlo estimator for the above function

$$
\lim_{N\to\infty} f_{MC}(x_0, y_0, z_0, N) = f(x_0, y_0, z_0) 
$$

$$
f_{MC}(x_0, y_0, z_0, N) = \frac{V}{N} \sum\limits_{[x, y, z]_1 \dots [x, y, z]_N} e^{-\frac{1}{2} \left( x - x_0 \right)^2} e^{-\frac{1}{2} \left( y - y_0 \right)^2} e^{-\frac{1}{2} \left( z - z_0 \right)^2} \sin \left( xy \right) \cos \left( z \right)
$$

where $ V = 11^3 $, as this is the total number of possible configurations

In [61]:
using Distributions, Random

In [46]:
function fMC(x0, y0, z0, N)
    
    draw = Array{Int}(undef, 3);
    draw_float_sample = Array{Float64}(undef, 1);
    Uniform_distribution = Uniform(0, 10);
    
    result = 0.0
   
    for i=1:N
        
          for i = 1:3
              rand!(Uniform_distribution, draw_float_sample)
              draw_float_sample[1] = round(Int64, draw_float_sample[1])
              draw[i] = Int(draw_float_sample[1])
          end
    
        result += exp(-(1/2)*(draw[1] - x0)^2)*exp(-(1/2)*(draw[2] - y0)^2)*exp(-(1/2)*(draw[3] - z0)^2)*sin(draw[1]*draw[2])*cos(draw[3])
        
    end
    
    result = result*11^3/N
        
    return result
    
end

fMC (generic function with 1 method)

In [85]:
@time fMC(2, 6, 10, 10^7)

  0.700662 seconds (2 allocations: 144 bytes)


0.6385288760178605

In [86]:
@time fMC(2, 6, 10, 10^7)

  0.713046 seconds (2 allocations: 144 bytes)


0.6606455083676828

In [87]:
@time fMC(2, 6, 10, 10^7)

  0.701753 seconds (2 allocations: 144 bytes)


0.6628532618085105

In [88]:
@time fMC(2, 6, 10, 10^7)

  0.707672 seconds (2 allocations: 144 bytes)


0.6448320741738877

$\textbf{The converge is slow}$: we need $10^7$ iterations to see a result stable up to the first significant digit. 

More importantly, $\textbf{the converges is biased}$, since the sum is highly oscillating and has several cancellations due to the negative terms.

# Changing variables

I can perform a trivial change of variables in order to sample from the interval $[0,1]$:

$$
X = \frac{x}{10}, \ Y = \frac{y}{10},\  Z = \frac{z}{10}
$$

The function changes as:

$$
f(x_0, y_0, z_0) = \sum\limits_{X \in [0,1]} \sum\limits_{Y \in [0,1]} \sum\limits_{Z \in [0,1]} e^{-\frac{1}{2} \left( 10*X - x_0 \right)^2} e^{-\frac{1}{2} \left( 10*Y - y_0 \right)^2} e^{-\frac{1}{2} \left( 10*Z - z_0 \right)^2} \sin \left( 100*XY \right) \cos \left( 10*Z \right)
$$

Since the original quantity was a sum and not a continuous integral, of course not all values $X \in [0,1]$ etc. should be considered, therefore it is necessary to round.

The MC estimator becomes:

$$
f_{MC}(x_0, y_0, z_0, N) = \frac{V}{N} \sum\limits_{[X, Y, Z]_1 \dots [X, Y, Z]_N} e^{-\frac{1}{2} \left( 10*X - x_0 \right)^2} e^{-\frac{1}{2} \left( 10*Y - y_0 \right)^2} e^{-\frac{1}{2} \left( 10*Z - z_0 \right)^2} \sin \left( 100*XY \right) \cos \left( 10*Z \right)
$$

In [67]:
function fMC_change_variable(x0, y0, z0, N)
    
    draw = Array{Int}(undef, 3);
    draw_float_sample = Array{Float64}(undef, 1);
    Uniform_distribution = Uniform(0, 1);
    
    result = 0.0
   
    for i=1:N
        
          for i = 1:3
              rand!(Uniform_distribution, draw_float_sample)
              draw_float_sample[1] = 10*draw_float_sample[1]
              draw_float_sample[1] = round(Int64, draw_float_sample[1])
              draw[i] = Int(draw_float_sample[1])
          end
    
        result += exp(-(1/2)*(draw[1] - x0)^2)*exp(-(1/2)*(draw[2] - y0)^2)*exp(-(1/2)*(draw[3] - z0)^2)*sin(draw[1]*draw[2])*cos(draw[3])
        
    end
    
    result = result*11^3/N
        
    return result
    
end

fMC_change_variable (generic function with 1 method)

In [71]:
@time fMC_change_variable(2, 6, 10, 10^7)

  0.709644 seconds (2 allocations: 144 bytes)


0.6556871341709338

## VEGAS

I write explicitly the equations considering a single axis for simplicity.

Let's start with a grid having $a = 0$, $b = 10$, $N_g = 10$ with all equal $\Delta x_i = \frac{b-a}{N_g} = 1$ for $i = 0 \dots 9$. 

$ x_0 = a $

$ x_1 = x_0 + \Delta x_0 $

$ x_2 = x_1 + \Delta x_1 $

$ \dots $

$ x_{N_g} = x_{N_g-1} + \Delta x_{N_g-1} = b $

In [144]:
a = 0;
b = 10;
N_g = 35;
Delta_x_i = (b-a)/(N_g);

We can use the $\textit{LinRange}$ function in julia, with last parameter equal to $N_g + 1$:

In [145]:
VEGAS_grid = collect(LinRange(a, b, N_g+1))

36-element Vector{Float64}:
  0.0
  0.2857142857142857
  0.5714285714285714
  0.8571428571428572
  1.1428571428571428
  1.4285714285714284
  1.7142857142857144
  2.0
  2.2857142857142856
  2.571428571428571
  2.8571428571428568
  3.142857142857143
  3.428571428571429
  ⋮
  6.857142857142858
  7.142857142857143
  7.428571428571429
  7.714285714285714
  8.0
  8.285714285714286
  8.571428571428571
  8.857142857142858
  9.142857142857142
  9.428571428571429
  9.714285714285714
 10.0

The first natural choice for the grid seems to be a linear spaced one:

In [146]:
VEGAS_spacing_map = [Delta_x_i for i in 1:1:N_g]

35-element Vector{Float64}:
 0.2857142857142857
 0.2857142857142857
 0.2857142857142857
 0.2857142857142857
 0.2857142857142857
 0.2857142857142857
 0.2857142857142857
 0.2857142857142857
 0.2857142857142857
 0.2857142857142857
 0.2857142857142857
 0.2857142857142857
 0.2857142857142857
 ⋮
 0.2857142857142857
 0.2857142857142857
 0.2857142857142857
 0.2857142857142857
 0.2857142857142857
 0.2857142857142857
 0.2857142857142857
 0.2857142857142857
 0.2857142857142857
 0.2857142857142857
 0.2857142857142857
 0.2857142857142857

The following function provides the map between $X \in [0,1]$ and $x \in [a, b]$

In [147]:
function x(X, N_g, VEGAS_grid) 
   
    i_X = Int(floor(X*N_g))
    
    delta_X = X*N_g - i_X
    
    if (i_X == N_g)
        return VEGAS_grid[i_X+1]
    else
        return VEGAS_grid[i_X+1] + VEGAS_spacing_map[i_X+1]*delta_X
    end
       
end

x (generic function with 2 methods)

In [154]:
@time x(0.3, N_g, VEGAS_grid)

  0.000006 seconds (5 allocations: 80 bytes)


2.9999999999999996

In [134]:
function fMC_change_variable(x0, y0, z0, N_ev, VEGAS_grid)
    
    draw = Array{Int}(undef, 3);
    draw_float_sample = Array{Float64}(undef, 1);
    Uniform_distribution = Uniform(0, 1);
    
    result = 0.0
   
    for i=1:N_ev
        
          for i = 1:3
              rand!(Uniform_distribution, draw_float_sample)
              println(draw_float_sample[1])
              println(x(draw_float_sample[1], 10, VEGAS_grid))
              draw_float_sample[1] = 10*draw_float_sample[1]
              draw_float_sample[1] = round(Int64, draw_float_sample[1])
              draw[i] = Int(draw_float_sample[1])
          end
    
        result += exp(-(1/2)*(draw[1] - x0)^2)*exp(-(1/2)*(draw[2] - y0)^2)*exp(-(1/2)*(draw[3] - z0)^2)*sin(draw[1]*draw[2])*cos(draw[3])
        
    end
    
    result = result*11^3/N_ev
        
    return result
    
end

fMC_change_variable (generic function with 2 methods)

In [133]:
@time fMC_change_variable(2, 6, 10, 10, VEGAS_grid)

0.4085216036121928
4.085216036121928
0.8477331487389025
8.477331487389025
0.6199880159372458
6.199880159372459
0.35445196528317535
3.5445196528317533
0.10134849871093499
1.0134849871093499
0.24454414407491332
2.445441440749133
0.2666846616216325
2.666846616216325
0.04735061192149992
0.4735061192149992
0.5815681027674933
5.815681027674932
0.6361533004189021
6.36153300418902
0.8517461076516576
8.517461076516575
0.4288354436182441
4.288354436182441
0.277262826664615
2.7726282666461497
0.2983859688462249
2.9838596884622492
0.36747183516618
3.6747183516617996
0.4988532595235574
4.988532595235574
0.5401475781133153
5.401475781133152
0.8283779450500972
8.283779450500973
0.374290046331168
3.7429004633116802
0.5131898959668818
5.131898959668818
0.5102018336052971
5.102018336052971
0.11620344636318491
1.1620344636318491
0.1290694782992653
1.290694782992653
0.7172691078369091
7.172691078369091
0.4411179156742414
4.411179156742414
0.569358207566085
5.69358207566085
0.46830484202479705
4.6830484202

0.002765682228838514

Now I want to estimate the following quantities:

$$
d^x_1 = \frac{1}{n^x_1} \sum_{X(x) \in \Delta x_1} \left( e^{-\frac{1}{2} \left( 10*X - x_0 \right)^2} e^{-\frac{1}{2} \left( 10*Y - y_0 \right)^2} e^{-\frac{1}{2} \left( 10*Z - z_0 \right)^2} \sin \left( 100*XY \right) \cos \left( 10*Z \right) \right)^2
$$

$$
d^x_2 = \frac{1}{n^x_2} \sum_{X(x) \in \Delta x_2} \left( e^{-\frac{1}{2} \left( 10*X - x_0 \right)^2} e^{-\frac{1}{2} \left( 10*Y - y_0 \right)^2} e^{-\frac{1}{2} \left( 10*Z - z_0 \right)^2} \sin \left( 100*XY \right) \cos \left( 10*Z \right) \right)^2
$$

$$
\dots
$$

$$
d^y_1 = \frac{1}{n^y_1} \sum_{Y(y) \in \Delta y_1} \left( e^{-\frac{1}{2} \left( 10*X - x_0 \right)^2} e^{-\frac{1}{2} \left( 10*Y - y_0 \right)^2} e^{-\frac{1}{2} \left( 10*Z - z_0 \right)^2} \sin \left( 100*XY \right) \cos \left( 10*Z \right) \right)^2
$$

etc. 

where $n^x_i \approx \frac{N_{ev}}{N_g}$ is the number of samples in interval $\Delta x_i$ 